# Fitting

In [1]:
import sys
sys.path.insert(0, '../../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from kraus_channels import KrausMap, DilutedKrausMap
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss, SpectrumDistance
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver
from quantum_channel import DilutedChannel, ChoiMapStatic
from quantum_tools import apply_unitary, measurement, resample, state_fidelity, generate_unitary
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits
from spam import SPAM, InitialState, POVMwQR as POVM, IdealPOVM, IdealInit, povm_fidelity, CorruptionMatrix
from synthetic_data import generate_spam_data, generate_map_data
from utils import saver, loader, train_val_split
from quantum_circuits import pqc_basic
from spectrum import channel_spectrum, mean_spacing


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def optimize_annulus_distance(spectrum_target = None, 
                              d = None, 
                              c_start = 0.5,
                              rank_start = 1,
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001,
                              verbose = True): 
    
    model_list = []
    distance_list = []
    counter = 0

    sigma = mean_spacing(spectrum_target)
    
    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_unitary(d), 
                                                          c=c_start, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = SpectrumDistance(sigma=sigma, k = None, remove_shift=True),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function_list = [SpectrumDistance(sigma=sigma, k = None)]),
                            )
        
        model.train(inputs = None,
                    targets = [spectrum_target],
                    inputs_val = [None],
                    targets_val = [[spectrum_target]],	
                    num_iter = num_iter,
                    verbose = verbose,
                   )

        c_start = model.channel.c.numpy()
        model_list.append(model)
        distance_list.append(model.logger.loss_list[0][-1])
        
        idx = np.argmin(distance_list)
        if distance_list[-1] > distance_list[idx]:
            counter += 1
        else:
            counter = 0
            
        if counter > 10:
            print("Done!")
            break

    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step, 
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        distance_best, model_best = optimize_annulus_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## L = 8

In [3]:
n = 4
d = 2**n

model = pkl.load(open("models/L=8.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.8,
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_8layer.model")

[0.09398802048180097]
[0.08213400017744374]
[0.07008870562823152]
[0.07188688599790932]
[0.07206309758815142]
[0.06863799497049827]
[0.06912090041687743]
[0.07532008538623668]
[0.07188184427094979]
[0.0636564375761388]
[0.05910825145324784]
[0.0668598187083335]
[0.06370695770971448]
[0.05858488731659492]
[0.05375024030760171]
[0.06636641979629566]
[0.06749068208723237]
[0.06794106024266648]
[0.0729047535610016]
[0.06737982684806833]
[0.06608444269904407]
[0.061277980962682915]
[0.0625384644309924]
[0.05958117085892789]
[0.0645198422129369]
[0.058241302057372574]
Done!


In [4]:
n = 4
d = 2**n

model = pkl.load(open("models/L=16.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.8,
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_16layer.model")

[0.08874753828696638]
[0.07677149989311852]
[0.06648793274602166]
[0.057751180741294185]
[0.0689845361989605]
[0.056134561479337555]
[0.056491147817214656]
[0.06292138704166754]
[0.054078516817047326]
[0.06270227365844663]
[0.06008803112979744]
[0.05671020151267196]
[0.05903585996049279]
[0.05110409659517551]
[0.05395653687322148]
[0.04875278959867892]
[0.05093986954694732]
[0.052059311642168496]
[0.05936407740631265]
[0.0513354662142894]
[0.054754003263585885]
[0.05194048170020883]
[0.04885528408545187]
[0.05535202879752747]
[0.06848556133195512]
[0.05608249653159175]
[0.04933098009629523]
Done!


In [5]:
n = 4
d = 2**n

model = pkl.load(open("models/L=24.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.65,
                                                      rank_start = 4, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_24layer.model")

[0.080240741376532]
[0.08354071608663544]
[0.07448171624654307]
[0.07394366809636796]
[0.07363112582635493]
[0.06792543742497506]
[0.06257479683353052]
[0.062400273814703096]
[0.06338455504964857]
[0.0661223988081088]
[0.06707712895209353]
[0.053551642464925824]
[0.04803959789944602]
[0.05969243111086485]
[0.061644790182637764]
[0.054408791100942946]
[0.06588246385042577]
[0.06220422965948323]
[0.061279689035111065]
[0.054809360589757425]
[0.054477621106669066]
[0.06409688565523364]
[0.061588684144813194]
[0.05122236353477351]
Done!


In [11]:
n = 4
d = 2**n

model = pkl.load(open("models/L=32.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.5,
                                                      rank_start = 4, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_32layer.model")

[0.06138874924196905]
[0.057836720685397566]
[0.05424060794977476]
[0.04878719181323059]
[0.048106406314210655]
[0.04710173538976397]
[0.04345796947879951]
[0.04611819333857226]
[0.041167627444073665]
[0.04888297949407508]
[0.050535673910983715]
[0.042856252306789196]
[0.03471325187403965]
[0.04657037027311743]
[0.049007881325227445]
[0.04310005015126728]
[0.03764065455025911]
[0.0388241647101962]
[0.05557506448021503]
[0.043545134433415594]
[0.04232596678368983]
[0.045820393761616104]
[0.04639542017497632]
[0.044731551516299956]
Done!


In [4]:
n = 4
d = 2**n

model = pkl.load(open("models/L=40.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.45,
                                                      rank_start = 16, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_40layer.model")

[0.05542813496948748]
[0.05852240742116135]
[0.05342591310027475]
[0.05486387542446177]
[0.05670320339018005]
[0.05097506171589629]
[0.05376499458810392]
[0.05752844767115374]
[0.056793527311084994]
[0.05775293043725541]
[0.0551051894454703]
[0.052384722144123416]
[0.05834235357192588]
[0.0545052175379826]
[0.05683697499056936]
[0.054343810470095205]
[0.051039610685769024]
Done!


In [10]:
n = 4
d = 2**n

model = pkl.load(open("models/L=56.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.3,
                                                      rank_start = 45, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_56layer.model")

[0.13720025636605565]
[0.13845099417397283]
[0.13982788611116123]
[0.13729951021013617]
[0.14198946432428494]
[0.14061569943983876]
[0.142201875581671]
[0.14150283344085707]
[0.14397300291561377]
[0.14182088348633992]
[0.14063475077879256]
[0.14355607450253519]
Done!


In [6]:
n = 4
d = 2**n

model = pkl.load(open("models/L=72.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.2,
                                                      rank_start = 46, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_72layer.model")

[0.033480843777202174]
[0.03400354651207803]
[0.035059353369193594]
[0.03192605417443033]
[0.03432338038770427]
[0.03771312215996063]
[0.03433952008401804]
[0.0362940396274557]
[0.03540323231921867]
[0.04196686530200859]
[0.030976527993425415]
[0.030318771799910396]
[0.03353802641841512]
[0.03441318249662981]
[0.037554874033426874]
[0.03413896616449141]
[0.03671797912012276]
[0.03464966435351742]
[0.03481359614201286]
[0.03625655414804309]
[0.041539635698455354]
[0.04237648418743577]
[0.03641372121743043]
Done!


In [7]:
n = 4
d = 2**n

model = pkl.load(open("models/L=88.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.15,
                                                      rank_start = 55, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_88layer.model")

[0.1513166963533356]
[0.154112602618251]
[0.15440322138988138]
[0.15319270852735878]
[0.15483970670446168]
[0.15567863345791288]
[0.03352169247410894]
[0.030985280491158798]
[0.0305075262838113]
[0.03589632326434114]
[0.02995287979209603]
[0.030057044242078487]
[0.03210574816317631]
[0.029742564833984617]
[0.028822995979534458]
[0.032010065082780065]
[0.034267517620180876]
[0.030368022467369027]
[0.036763493246830536]
[0.04007803507819235]
[0.03565745482602524]
[0.03011420126935005]
[0.03435244275766759]
[0.03201531502736052]
[0.02720004000027723]
[0.02817026362692454]
[0.037015601252134586]
[0.034596841593730596]
[0.030163073632614945]
[0.03326626837560456]
[0.03816753919320311]
[0.03374729555107159]
[0.02767474402738411]
[0.031664569080693816]
[0.033255714796884606]
[0.03301387184936118]
Done!


In [8]:
n = 4
d = 2**n

model = pkl.load(open("models/L=104.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.12,
                                                      rank_start = 82, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_104layer.model")

[0.025315336448043846]
[0.02457854795046839]
[0.0233851900300652]
[0.02180452512036106]
[0.024176980339748022]
[0.02306347546710044]
[0.02530942028114777]
[0.022638908832804477]
[0.026036733500680795]
[0.023960675856660008]
[0.024929801368972354]
[0.02435591240540405]
[0.0269822229135097]
[0.02394633946648478]
[0.02449157080140843]
Done!


In [9]:
n = 4
d = 2**n

model = pkl.load(open("models/L=120.model", "rb"))
spectrum_target = channel_spectrum(model[0].channel)

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum_target, 
                                                      d = d,
                                                      c_start = 0.07,
                                                      rank_start = 79, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], "models/" + "diluted_fitting_KL_4qubit_120layer.model")

[0.045575118353355815]
[0.039842034661013825]
[0.03695614077536602]
[0.03694110141570074]
[0.039722105385324474]
[0.035302528230694546]
[0.039703813197904086]
[0.03622751851385629]
[0.03302616521959352]
[0.03794278091647511]
[0.03531290087098417]
[0.038955293282024]
[0.035191494315633634]
[0.03556883029524202]
[0.03832203185634729]
[0.03406279288882104]
[0.02991807919215854]
[0.028568915778292375]
[0.026971600572008328]
[0.02788048942742563]
[0.027428699466876122]
[0.029128186586825075]
[0.033393969063249984]
[0.029573416065586805]
[0.029972901667626284]
[0.02890855645850582]
[0.03235594542564986]
[0.02918442071936564]
[0.031066235728737845]
[0.02626702652341749]
[0.024095865938105837]
[0.02928280049565321]
[0.02948989100409702]
[0.025594421104149667]
[0.025078728301794834]
[0.02700694110612796]
[0.029203285608894666]
[0.028076185229001044]
[0.029051732997851798]
[0.029839232546499672]
[0.024043995655399983]
[0.026281284292719184]
[0.028006854986850432]
[0.026376341856020807]
[0.025923